**MI6 Debrief:**

> Should be have a flip flag for each metric? To identify good/bad *Solved, negative coefficients will account for that*

> Should we fill 0 to NA data? or fill it up by mean? *Used both*

> Missing data is an issue, regression wont run unless complete data

> Should we remove 2020 data from control variables *Have done, data only upto 2019 now*

> Have not included cuurency data yet, was not clear.*Phase 2*

> Inflation cleaned .xlsx is the latest version right?

**007, I'm your new Quartermaster.**

In [3]:
import pandas as pd
import numpy as np
import datetime as dt
from functools import reduce
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import boxcox
from scipy.stats import yeojohnson
from statsmodels.stats.diagnostic import het_white
from statsmodels.compat import lzip
from statsmodels.stats.diagnostic import acorr_breusch_godfrey as bg

**What makes you think this is my first time?**

In [4]:
import warnings
warnings.filterwarnings("ignore")

**A gun and a radio...not exactly Christmas, is it?**

In [5]:
Dp_FDI_wide = pd.read_csv("Dp_FDI.csv")
#Dp_FDI_wide = pd.read_csv("FDI_yoy.csv")
Dp_PE_wide = pd.read_csv("Dp_PENew.csv")
Dp_Curr_wide = pd.read_csv("Dp_Curr.csv")
Dp_IntRate_wide = pd.read_csv("Dp_IntRate.csv")
Ind_TerrIdx = pd.read_csv("Ind_TerrIdx.csv")
Ind_TerrRaw = pd.read_csv("Ind_Terr.csv")
Cn_LaborForce_wide = pd.read_csv("Cn_LaborForce.csv")
Cn_GDPpc_wide = pd.read_csv("Cn_GDPpc.csv")
Cn_EODB_wide = pd.read_csv("EODBPercentiled.csv")
Cn_DomCrdt_wide = pd.read_csv("Cn_DomCrdt.csv")
Cn_FixedCapForm_wide = pd.read_csv("Cn_FixedCapForm.csv")
Cn_Infl_wide = pd.read_csv("Cn_Infl.csv")
Cn_IntRateL_wide = pd.read_csv("Cn_IntRateL.csv")
Cn_CurrStd_wide = pd.read_csv("Cn_CurrStd.csv")

In [6]:
Dp_FDI = pd.melt(Dp_FDI_wide,id_vars = ("Country Name"), var_name = "Year", value_name = "FDI")
Dp_PE = pd.melt(Dp_PE_wide,id_vars = ("Country Name"), var_name = "Year", value_name = "PE")
Dp_Curr = pd.melt(Dp_Curr_wide,id_vars = ("Year"), var_name = "Country Name", value_name = "Currency")
Dp_IntRate = pd.melt(Dp_IntRate_wide,id_vars = ("Country Name"), var_name = "Year", value_name = "IntRate")
Cn_LaborForce = pd.melt(Cn_LaborForce_wide,id_vars = ("Country Name"), var_name = "Year", value_name = "Lab")
Cn_GDPpc = pd.melt(Cn_GDPpc_wide,id_vars = ("Country Name"), var_name = "Year", value_name = "GDPpc")
Cn_EODB = pd.melt(Cn_EODB_wide,id_vars = ("Country Name"), var_name = "Year", value_name = "EODB")
Cn_DomCrdt = pd.melt(Cn_DomCrdt_wide,id_vars = ("Country Name"), var_name = "Year", value_name = "DomCrdt")
Cn_FixedCapForm = pd.melt(Cn_FixedCapForm_wide,id_vars = ("Country Name"), var_name = "Year", value_name = "FixedCapForm")
Cn_Infl = pd.melt(Cn_Infl_wide,id_vars = ("Country Name"), var_name = "Year", value_name = "Inf")
Cn_IntRateL = pd.melt(Cn_IntRateL_wide,id_vars = ("Country Name"), var_name = "Year", value_name = "IntRateLend")
Cn_CurrStd = pd.melt(Cn_CurrStd_wide,id_vars = ("Year"), var_name = "Country Name", value_name = "CurStd")[["Country Name", "Year", "CurStd"]]
Cn_CurrStd["Year"] = Cn_CurrStd["Year"].apply(str)

In [7]:
# Merging control variables into one dataset
dfs = [Cn_LaborForce, Cn_GDPpc, Cn_EODB, Cn_DomCrdt,Cn_FixedCapForm, Cn_Infl, Cn_IntRateL,Cn_CurrStd]
Control_Matrix = reduce(lambda df_left,df_right: pd.merge(df_left, df_right, 
                        left_on=["Country Name","Year"], right_on = ["Country Name","Year"], how='outer'), dfs)
Control_Matrix["Year"] = pd.to_numeric(Control_Matrix["Year"])
Control_Matrix = Control_Matrix[Control_Matrix["Year"]<2020]
Control_Matrix["Year"] = Control_Matrix["Year"].apply(str)

**Be careful what you wish for.**

In [8]:
#1a. Terrorism Incidents Data as many gaps. So creating dummy '0' datapoints for completeness
Ind_Terr_Wide = Ind_TerrRaw.pivot(index="Year", columns="Country Name", values= "Incidents").reset_index(drop=False).fillna(0)
Ind_Terr = pd.melt(Ind_Terr_Wide,id_vars = ("Year"), var_name = "Country Name", value_name = "Incidents")

#1b. Including Terrorism Incidents Lag Variables
for i in range(0,len(Ind_Terr["Country Name"].unique())):
    country = Ind_Terr["Country Name"].unique()[i]
    Ter = Ind_Terr[Ind_Terr["Country Name"] == country]
    Ter["IncidentLag_5"] = Ter["Incidents"].shift(5)
    if i == 0:
        Ind_Terr_Ref = Ter
    else:
        Ind_Terr_Ref = pd.concat([Ind_Terr_Ref, Ter])
Ind_Terr_Ref.dropna(inplace=True)

#2a. Including Terrorism Index Lag Variables
for i in range(0,len(Ind_TerrIdx["Country Name"].unique())):
    country = Ind_TerrIdx["Country Name"].unique()[i]
    Ter = Ind_TerrIdx[Ind_Terr["Country Name"] == country]
    Ter["TerrorIdxLag_5"] = Ter["TerrIdx"].shift(5)
    if i == 0:
        Ind_TerrIdx_Ref = Ter
    else:
        Ind_TerrIdx_Ref = pd.concat([Ind_TerrIdx_Ref, Ter])
Ind_TerrIdx_Ref.dropna(inplace=True)

In [9]:
#Replcaing Nan values in Control Matrix with mean()
for i in range(0,len(Control_Matrix["Country Name"].unique())):
    country = Control_Matrix["Country Name"].unique()[i]
    Cntrl = Control_Matrix[Control_Matrix["Country Name"] == country]
    Cntrl.loc[:, ["Lab", "GDPpc", "EODB", "DomCrdt","FixedCapForm", "Inf", "IntRateLend"]] = Cntrl.loc[:, ["Lab", "GDPpc", "EODB", "DomCrdt","FixedCapForm", "Inf", "IntRateLend"]].apply(lambda x: x.fillna(x.mean()),axis=0)
    if i == 0:
        Control_Matrix_Ref = Cntrl
    else:
        Control_Matrix_Ref = pd.concat([Control_Matrix_Ref, Cntrl])
Control_Matrix_Ref.dropna(inplace=True)

In [14]:
#Combine and export database to check
Ind_Terr_Ref["Year"] = Ind_Terr_Ref["Year"].apply(str)
Ind_TerrIdx_Ref["Year"] = Ind_TerrIdx_Ref["Year"].apply(str)
Dp_Curr["Year"] = Dp_Curr["Year"].apply(str)
Dp_PE["Year"] = Dp_PE["Year"].apply(str)
df2 = [Dp_FDI, Dp_PE, Dp_Curr, Dp_IntRate, Ind_Terr_Ref, Ind_TerrIdx_Ref, Control_Matrix_Ref]
#df2 = [Dp_PE, Ind_Terr_Ref, Ind_TerrIdx_Ref, Control_Matrix_Ref]
MergedFile = reduce(lambda df_left,df_right: pd.merge(df_left, df_right, 
                        left_on=["Country Name","Year"], right_on = ["Country Name","Year"], how='outer'), df2)
print("Before dropping NA",MergedFile.shape)
MergedFile.dropna(inplace = True)
print("After dropping NA",MergedFile.shape)

Before dropping NA (2143, 18)
After dropping NA (276, 18)


In [15]:
MergedFile.describe()

,FDI,PE,Currency,IntRate,Incidents,IncidentLag_5,TerrIdx,TerrorIdxLag_5,Lab,GDPpc,EODB,DomCrdt,FixedCapForm,Inf,IntRateLend,CurStd
count,276.000000,276.000000,276.000000,276.000000,276.000000,276.000000,276.000000,276.000000,276.000000,276.000000,276.000000,276.000000,276.000000,276.000000,276.000000,276.000000
mean,4.249007,0.109674,0.025151,3.160425,61.355072,38.793478,2.368264,2.077217,60.010942,1.655723,6.628623,88.837443,22.543330,4.172886,7.403909,86.335439
std,9.574844,13.481869,0.095616,3.096056,208.575752,168.650572,2.344087,2.151749,8.261398,2.747049,2.592425,48.041497,4.325821,5.809303,5.742454,308.227038
min,-40.329928,-106.550000,-0.143371,-7.773405,0.000000,0.000000,0.000000,0.000000,39.250000,-7.226150,1.000000,12.877721,10.873349,-2.854865,0.500000,0.000210
25%,1.310527,-1.967500,-0.029797,1.464118,0.000000,0.000000,0.286000,0.229000,54.145000,0.348819,5.000000,39.142532,19.577264,1.264504,4.142313,0.120589
50%,2.813322,-0.100000,0.008704,3.167835,2.000000,1.500000,1.632500,1.413500,60.905000,1.630426,7.000000,91.170622,22.651027,2.830401,6.574542,1.142346
75%,4.879278,2.612500,0.057531,4.892442,15.250000,10.250000,4.125250,3.444750,65.592500,3.580429,9.000000,127.440848,25.138218,5.091756,9.352292,5.130772
max,86.589086,107.660000,0.684335,12.670410,2215.000000,2215.000000,8.843000,8.366000,77.900000,11.144209,10.000000,192.474207,37.286513,50.622568,67.254167,1901.200054


In [11]:
addvalue = abs(int(min(list(MergedFile.min()[2:]))/5 - 1)*5)

In [13]:
MergedFile.min()

Country Name      Argentina
Year                   2001
PE                  -106.55
Incidents               0.0
IncidentLag_5           0.0
TerrIdx                 0.0
TerrorIdxLag_5          0.0
Lab                   39.25
GDPpc              -7.22615
EODB                    1.0
DomCrdt            9.682518
FixedCapForm      10.873349
Inf               -2.854865
IntRateLend             0.5
CurStd              0.00021
dtype: object

In [12]:
addvalue

110

In [47]:
transform_type = "log"

In [48]:
if transform_type == "sqrt":
    MergedFile.iloc[:,2:] = MergedFile.iloc[:,2:]+addvalue
    MergedFile.iloc[:,2:] = np.sqrt(MergedFile.iloc[:,2:])
elif transform_type == "log":
    MergedFile.iloc[:,2:] = MergedFile.iloc[:,2:]+addvalue
    MergedFile.iloc[:,2:] = np.log(MergedFile.iloc[:,2:])

MergedFile

**Take the bloody shot!**

In [49]:
result = sm.OLS(MergedFile[["PE"]], sm.add_constant(MergedFile[["TerrIdx","IncidentLag_5", "Lab", "GDPpc", "EODB", "DomCrdt","FixedCapForm", "Inf", "IntRateLend","CurStd"]]), missing = "drop").fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                     PE   R-squared:                       0.014
Model:                            OLS   Adj. R-squared:                 -0.018
Method:                 Least Squares   F-statistic:                    0.4326
Date:                Fri, 29 Oct 2021   Prob (F-statistic):              0.930
Time:                        00:16:28   Log-Likelihood:                 36.774
No. Observations:                 326   AIC:                            -51.55
Df Residuals:                     315   BIC:                            -9.893
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             5.4641      7.635      0.716

> White's test for heteroskedacity

In [50]:
wtest = het_white(result.resid, result.model.exog)
labels = ['Lagrange Multiplier statistic:', 'LM test\'s p-value:', 'F-statistic:', 'F-test\'s p-value:']
lzip(labels, wtest)

[('Lagrange Multiplier statistic:', 12.794910103350116),
 ("LM test's p-value:", 0.999999999999786),
 ('F-statistic:', 0.16340615802344896),
 ("F-test's p-value:", 0.9999999999999826)]

> 2nd order autocorrelation

In [51]:
print(bg(result, nlags=2))

(0.0634620162699504, 0.9687671375130749, 0.03047159305203399, 0.9699908631850741)


> 3rd order autocorrelation

In [52]:
print(bg(result, nlags=3))

(0.06666828602640185, 0.9955122546794793, 0.021272760629535512, 0.9957840100138134)


**Where the hell have you been? Enjoying death.**

In [65]:
result = sm.OLS(MergedFile[["PE"]], sm.add_constant(MergedFile[["Incidents","IncidentLag_5","TerrIdx","TerrorIdxLag_5","Lab", "GDPpc", "EODB", "DomCrdt", "Inf", "IntRateLend","CurStd"]]), missing = "drop").fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                     PE   R-squared:                       0.022
Model:                            OLS   Adj. R-squared:                 -0.017
Method:                 Least Squares   F-statistic:                    0.5576
Date:                Wed, 27 Oct 2021   Prob (F-statistic):              0.862
Time:                        20:15:32   Log-Likelihood:                -434.65
No. Observations:                 288   AIC:                             893.3
Df Residuals:                     276   BIC:                             937.3
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const              0.9715      0.698      1.

In [66]:
result = sm.OLS(MergedFile[["Currency"]], sm.add_constant(MergedFile[["Incidents","IncidentLag_5","TerrIdx","Lab", "GDPpc", "EODB", "DomCrdt", "Inf", "IntRateLend","CurStd"]]), missing = "drop").fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:               Currency   R-squared:                       0.310
Model:                            OLS   Adj. R-squared:                  0.286
Method:                 Least Squares   F-statistic:                     12.47
Date:                Wed, 27 Oct 2021   Prob (F-statistic):           7.20e-18
Time:                        20:15:50   Log-Likelihood:                 324.28
No. Observations:                 288   AIC:                            -626.6
Df Residuals:                     277   BIC:                            -586.3
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             0.0042      0.049      0.086

In [67]:
result = sm.OLS(MergedFile[["IntRate"]], sm.add_constant(MergedFile[["Incidents","IncidentLag_5","TerrIdx","Lab", "GDPpc", "EODB", "DomCrdt","CurStd"]]), missing = "drop").fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                IntRate   R-squared:                       0.100
Model:                            OLS   Adj. R-squared:                  0.075
Method:                 Least Squares   F-statistic:                     3.892
Date:                Wed, 27 Oct 2021   Prob (F-statistic):           0.000226
Time:                        20:15:55   Log-Likelihood:                -734.22
No. Observations:                 288   AIC:                             1486.
Df Residuals:                     279   BIC:                             1519.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             2.4888      1.934      1.287

In [69]:
MergedFile

,Country Name,Year,FDI,Incidents,IncidentLag_1,IncidentLag_2,IncidentLag_3,IncidentLag_4,IncidentLag_5,IncidentLag_6,TerrIdx,Lab,GDPpc,EODB,DomCrdt,Inf,IntRate,CurStd
0,Argentina,2000,3.665791,0.0,1.0,0.0,11.0,19.0,16.0,14.0,NaN,61.85,-1.874936,113.785714,23.894734,1.037287,27.836417,0.000237
1,Australia,2000,3.586745,1.0,0.0,6.0,4.0,5.0,5.0,9.0,NaN,63.37,2.701329,11.214286,87.735759,2.579702,7.716667,0.187072
2,Bolivia,2000,8.768250,0.0,0.0,0.0,4.0,1.0,2.0,8.0,NaN,70.79,0.544855,148.785714,49.293750,5.195408,34.595833,0.430783
3,Canada,2000,9.171024,5.0,2.0,4.0,2.0,1.0,5.0,4.0,NaN,65.27,3.945219,12.785714,73.987738,4.593832,7.270833,0.060662
4,Dominican Republic,2000,4.069001,0.0,0.0,0.0,12.0,3.0,0.0,7.0,NaN,59.53,3.079985,101.071429,23.570357,7.207763,26.799167,0.995494
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1835,Gambia,2019,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1836,Macedonia,2019,NaN,0.0,0.0,1.0,0.0,4.0,3.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1837,Slovenia,2019,3.156400,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1838,Kyrgyzstan,2019,NaN,0.0,1.0,3.0,4.0,3.0,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
f= open("Regression_droppingallna.txt","w+")
for dp in ["FDI","PE","Currency","IntRate"]:
    if dp == "IntRate":
        result = sm.OLS(MergedFile[[dp]], sm.add_constant(MergedFile[["TerrIdx","IncidentLag_5","Lab", "GDPpc", "EODB", "DomCrdt","CurStd"]]), missing = "drop").fit()
    else:
        result = sm.OLS(MergedFile[[dp]], sm.add_constant(MergedFile[["TerrIdx","IncidentLag_5","Lab", "GDPpc", "EODB", "DomCrdt", "Inf", "IntRateLend","CurStd"]]), missing = "drop").fit()
    print(result.summary())
    f.write(str(result.summary()))
f.close()


In [53]:
MergedFile.iloc[:,2] = boxcox(MergedFile.iloc[:,2])[0]
MergedFile.iloc[:,3] = boxcox(MergedFile.iloc[:,3])[0]
MergedFile.iloc[:,4] = boxcox(MergedFile.iloc[:,4])[0]
MergedFile.iloc[:,5] = boxcox(MergedFile.iloc[:,5])[0]
MergedFile.iloc[:,6] = boxcox(MergedFile.iloc[:,6])[0]
MergedFile.iloc[:,7] = boxcox(MergedFile.iloc[:,7])[0]
MergedFile.iloc[:,8] = boxcox(MergedFile.iloc[:,8])[0]
MergedFile.iloc[:,9] = boxcox(MergedFile.iloc[:,9])[0]
MergedFile.iloc[:,10] = boxcox(MergedFile.iloc[:,10])[0]
MergedFile.iloc[:,11] = boxcox(MergedFile.iloc[:,11])[0]
MergedFile.iloc[:,12] = boxcox(MergedFile.iloc[:,12])[0]
MergedFile.iloc[:,13] = boxcox(MergedFile.iloc[:,13])[0]

In [59]:
MergedFile.iloc[:,2] = yeojohnson(MergedFile.iloc[:,2])[0]
MergedFile.iloc[:,3] = yeojohnson(MergedFile.iloc[:,3])[0]
MergedFile.iloc[:,4] = yeojohnson(MergedFile.iloc[:,4])[0]
MergedFile.iloc[:,5] = yeojohnson(MergedFile.iloc[:,5])[0]
MergedFile.iloc[:,6] = yeojohnson(MergedFile.iloc[:,6])[0]
MergedFile.iloc[:,7] = yeojohnson(MergedFile.iloc[:,7])[0]
MergedFile.iloc[:,8] = yeojohnson(MergedFile.iloc[:,8])[0]
MergedFile.iloc[:,9] = yeojohnson(MergedFile.iloc[:,9])[0]
MergedFile.iloc[:,10] = yeojohnson(MergedFile.iloc[:,10])[0]
MergedFile.iloc[:,11] = yeojohnson(MergedFile.iloc[:,11])[0]
MergedFile.iloc[:,12] = yeojohnson(MergedFile.iloc[:,12])[0]
MergedFile.iloc[:,13] = yeojohnson(MergedFile.iloc[:,13])[0]

In [ ]:
bg(result, nlags=5)

In [ ]:
bg(result, nlags=5)

In [62]:
countries = set(Ind_Terr["Country Name"].unique()).intersection(Control_Matrix["Country Name"].unique()).intersection(Dp_FDI["Country Name"].unique())

Y = intercept + α(Incident t) + α1(Incident t-1) + α2(Incident t-2) + α3(Incident t-3) + α4(Incident t-4) + α5(Incident t-5) + α6(Incident t-6) + αc1(Control Labour Force%) + ac2(Control GDPpc) + ac3(Control EODB) + ac4(Control Domestic Crdt Sprd) + ac5(Control Inflation) +ac6(Control Interest Rate) 

New Version

In [ ]:
#Run for specific Country
countries = ["Japan"]
for country in countries:
    print("Running", country)
    Cntrl = Control_Matrix[Control_Matrix["Country Name"] == country]
    #Filling NaN Values with mean()
    Cntrl.loc[:, ["Lab", "GDPpc", "EODB", "DomCrdt", "Inf", "IntRate"]] = Cntrl.loc[:, ["Lab", "GDPpc", "EODB", "DomCrdt", "Inf", "IntRate"]].apply(lambda x: x.fillna(x.mean()),axis=0)
    Ter = Ind_Terr[Ind_Terr["Country Name"] == country]
    cols = []
    for lag in range(1, lags + 1): 
        col = f"IncidentLag_{lag}"
        Ter[col] = Ter["Incidents"].shift(lag)
        cols.append(col)
    Ter.dropna(inplace = True)
    Dp = Dp_FDI[Dp_FDI["Country Name"] == country]
    Ind = pd.merge(Ter, Cntrl,  how='outer', left_on=["Country Name","Year"], right_on = ["Country Name","Year"])
    Dp.fillna(0, inplace = True)
    Ind.fillna(0, inplace = True)
    result = sm.OLS(Dp[["FDI"]].reset_index(drop=True), sm.add_constant(Ind[["Incidents","IncidentLag_1", "IncidentLag_2", "IncidentLag_3", "IncidentLag_4", "IncidentLag_5", "IncidentLag_6","Lab", "GDPpc", "EODB", "DomCrdt", "Inf", "IntRate","CurStd"]])).fit()
    print(result.summary())

Old Below

In [75]:
FinalTable = pd.DataFrame(index = range(0,len(countries)))
FinalTable["Country"] = list(countries)
FinalTable["intercept"] = None
FinalTable["a"] = None
FinalTable["a1"] = None
FinalTable["a2"] = None
FinalTable["a3"] = None
FinalTable["a4"] = None
FinalTable["a5"] = None
FinalTable["a6"] = None
FinalTable["ac1"] = None
FinalTable["ac2"] = None
FinalTable["ac3"] = None
FinalTable["ac4"] = None
FinalTable["ac5"] = None
FinalTable["ac6"] = None

In [76]:
for country in countries:
    print("Running", country)
    Cntrl = Control_Matrix[Control_Matrix["Country Name"] == country]
    #Filling NaN Values with mean()
    Cntrl.loc[:, ["Lab", "GDPpc", "EODB", "DomCrdt", "Inf", "IntRate"]] = Cntrl.loc[:, ["Lab", "GDPpc", "EODB", "DomCrdt", "Inf", "IntRate"]].apply(lambda x: x.fillna(x.mean()),axis=0)
    Ter = Ind_Terr[Ind_Terr["Country Name"] == country]
    cols = []
    for lag in range(1, lags + 1): 
        col = f"IncidentLag_{lag}"
        Ter[col] = Ter["Incidents"].shift(lag)
        cols.append(col)
    Ter.dropna(inplace = True)
    Dp = Dp_FDI[Dp_FDI["Country Name"] == country]
    Ind = pd.merge(Ter, Cntrl,  how='outer', left_on=["Country Name","Year"], right_on = ["Country Name","Year"])
    Dp.fillna(0, inplace = True)
    Ind.fillna(0, inplace = True)
    #print(len(Cntrl))
    #print(len(Ind))
    #print(len(Dp))
    Model = LinearRegression().fit(Ind[["IncidentLag_1", "IncidentLag_2", "IncidentLag_3", "IncidentLag_4", "IncidentLag_5"
                                        , "IncidentLag_6","Lab", "GDPpc", "EODB", "DomCrdt", "Inf", "IntRate"]],Dp[["FDI"]])
    #print(country, "Model Coefficients:", Model.coef_ , "Model Intercept:", Model.intercept_)
    FinalTable["intercept"][FinalTable["Country"]==country] = Model.intercept_
    FinalTable["a"][FinalTable["Country"]==country] = Model.coef_[0,0]
    FinalTable["a1"][FinalTable["Country"]==country] = Model.coef_[0,0]
    FinalTable["a2"][FinalTable["Country"]==country] = Model.coef_[0,1]
    FinalTable["a3"][FinalTable["Country"]==country] = Model.coef_[0,2]
    FinalTable["a4"][FinalTable["Country"]==country] = Model.coef_[0,3]
    FinalTable["a5"][FinalTable["Country"]==country] = Model.coef_[0,4]
    FinalTable["a6"][FinalTable["Country"]==country] = Model.coef_[0,5]
    FinalTable["ac1"][FinalTable["Country"]==country] = Model.coef_[0,0]
    FinalTable["ac2"][FinalTable["Country"]==country] = Model.coef_[0,0]
    FinalTable["ac3"][FinalTable["Country"]==country] = Model.coef_[0,0]
    FinalTable["ac4"][FinalTable["Country"]==country] = Model.coef_[0,0]
    FinalTable["ac5"][FinalTable["Country"]==country] = Model.coef_[0,0]
    FinalTable["ac6"][FinalTable["Country"]==country] = Model.coef_[0,0]

Running Japan


In [84]:
FinalTable.to_csv("Regression_Output_v1.0.csv", index = False)